In [16]:
from telethon.sync import TelegramClient
import pandas as pd
from hazm import Normalizer, WordTokenizer, stopwords_list, Lemmatizer
import re
import emoji
from gensim.models.phrases import Phrases, Phraser
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN



api_id = 21021126          
api_hash = '63b67d261fce0b5167175fc84ceb9ca2'

client = TelegramClient("session_persian", api_id, api_hash)


In [17]:
await client.start()

In [18]:
# Scraping for Data from a Telgram channel
channel_username = 'IranintlTV'   
limit = 10000                     
keywords = ['جنگ']  

all_messages = []

async with client:
    async for message in client.iter_messages(channel_username, limit=limit):
        if message.message:
            text = message.message
            if any(keyword in text for keyword in keywords):
                all_messages.append({
                    "date": message.date,
                    "text": text
                })

df = pd.DataFrame(all_messages)
df.to_csv("../data/data.csv", index=False, encoding="utf-8-sig")
df.head()

,date,text
0,2025-07-22 21:18:22+00:00,غلامحسین محسنی اژه‌ای، رییس قوه قضاییه جمهوری ...
1,2025-07-22 20:09:26+00:00,یسرائیل کاتز، وزیر دفاع اسرائیل، در جلسه‌ای با...
2,2025-07-22 19:45:10+00:00,شهرام خلدی، پژوهشگر تاریخ خاورمیانه و روابط بی...
3,2025-07-22 19:44:26+00:00,سازمان عفو بین‌الملل اعلام کرد حمله اسرائیل به...
4,2025-07-22 18:46:13+00:00,حسین آقایی، عضو تحریریه ایران‌اینترنشنال، دربا...


In [19]:
# Cleaning the text for the embedder and the tokenizer
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = emoji.replace_emoji(text, replace='')         
    text = re.sub(r"@\w+", "", text)                     
    text = re.sub(r"http\S+|www\S+", "", text)           
    text = re.sub(r"\s+", " ", text).strip()             
    return text

In [20]:
df = pd.read_csv("../data/data.csv")
df["clean_text"] = df["text"].map(clean_text)
df[['text', 'clean_text']].head()

,text,clean_text
0,غلامحسین محسنی اژه‌ای، رییس قوه قضاییه جمهوری ...,غلامحسین محسنی اژه‌ای، رییس قوه قضاییه جمهوری ...
1,یسرائیل کاتز، وزیر دفاع اسرائیل، در جلسه‌ای با...,یسرائیل کاتز، وزیر دفاع اسرائیل، در جلسه‌ای با...
2,شهرام خلدی، پژوهشگر تاریخ خاورمیانه و روابط بی...,شهرام خلدی، پژوهشگر تاریخ خاورمیانه و روابط بی...
3,سازمان عفو بین‌الملل اعلام کرد حمله اسرائیل به...,سازمان عفو بین‌الملل اعلام کرد حمله اسرائیل به...
4,حسین آقایی، عضو تحریریه ایران‌اینترنشنال، دربا...,حسین آقایی، عضو تحریریه ایران‌اینترنشنال، دربا...


In [34]:
# A light tokenizer for generating bigrams 
light_stops = {"است", "می", "شود"}
normalizer = Normalizer(
    persian_numbers=True,
    remove_diacritics=True,
)
tokenizer = WordTokenizer()
lemmatizer  = Lemmatizer()

def persian_tokenizer_light(text: str) -> list[str]:
    if not isinstance(text, str):
        return []
    text = normalizer.normalize(text)
    text = re.sub(r"[^\u0600-\u06FF0-9\s]", " ", text)

    tokens = [
        lemmatizer.lemmatize(tok)
        for tok in tokenizer.tokenize(text)
        if tok not in light_stops and len(tok) > 1
    ]
    cleaned = []
    for t in tokens:
        if "#" in t:           
            continue
        if t.isdigit():        
            continue
        if len(t) < 3:         
            continue
        cleaned.append(t)
    return cleaned

In [35]:
class SentenceIterator:
    def __init__(self, series):
        self.series = series  
    def __iter__(self):
        for txt in self.series:
            yield persian_tokenizer_light(txt)

sentences = SentenceIterator(df['clean_text'])

In [ ]:
# Generating bigrams 
cached = list(tqdm(sentences, desc="Tokenising"))

bigram_model = Phrases(
    cached,
    min_count=10,
    threshold=5,
    #scoring='npmi',
    delimiter='_',      
)
bigram_phraser = Phraser(bigram_model)
bigram_phraser.save("persian_bigram_phraser.gensim")


sentences_bigrams = (bigram_phraser[sent] for sent in cached)   

Tokenising: 1504it [00:01, 1215.98it/s]


In [38]:
# Final Tokenization
base_stops = set(stopwords_list())
extra_stops = extra_stops = {
    "است","می","شود","iranintltv","http","https",
    "jpg","png","video","photo","کانال","تلگرام","ای","های","ها",
    "ایراناینترنشنال", "اینترنشنال", "ویدیو", "کانال"
}

stops = base_stops | extra_stops

bigram_phraser  = Phraser.load("persian_bigram_phraser.gensim")

def persian_tokenizer(text: str) -> list[str]:
    if not isinstance(text, str):
        return []
    
    tokens = persian_tokenizer_light(text)          
    tokens = bigram_phraser[tokens]                 

    tokens = [
        t for t in tokens
        if t not in stops       
        and len(t) >= 3
        and not t.isdigit()
    ]

    return tokens

df['tokens'] = df['clean_text'].apply(persian_tokenizer)
df[['clean_text', 'tokens']].head()

,clean_text,tokens
0,غلامحسین محسنی اژه‌ای، رییس قوه قضاییه جمهوری ...,"[غلامحسین, محسنی_اژه, رییس, قوه_قضاییه, جمهوری..."
1,یسرائیل کاتز، وزیر دفاع اسرائیل، در جلسه‌ای با...,"[یسرائیل, کاتز, وزیر_دفاع, اسرائیل, جلسه, فرما..."
2,شهرام خلدی، پژوهشگر تاریخ خاورمیانه و روابط بی...,"[شهرام_خلدی, پژوهشگر_تاریخ, خاورمیانه, روابط_ب..."
3,سازمان عفو بین‌الملل اعلام کرد حمله اسرائیل به...,"[سازمان, عفو_بین, الملل, اعلام, حمله, اسرائیل,..."
4,حسین آقایی، عضو تحریریه ایران‌اینترنشنال، دربا...,"[حسین, آقایی, عضو_تحریریه, ایران_اینترنشنال, ع..."


In [39]:
sample_doc = df['clean_text'].iloc[0]
print("Original:", sample_doc[:120], "...")
print("Tokens:", persian_tokenizer_light(sample_doc)[:15])
print("With bigrams:", bigram_phraser[persian_tokenizer_light(sample_doc)][:15])

Original: غلامحسین محسنی اژه‌ای، رییس قوه قضاییه جمهوری اسلامی، در پاسخ به سوال علی رضوانی، بازجو-خبرنگار صداو‌سیما، درباره نقش ره ...
Tokens: ['غلامحسین', 'محسنی', 'اژه', 'رییس', 'قوه', 'قضاییه', 'جمهوری', 'اسلامی', 'پاسخ', 'سوال', 'علی', 'رضوانی', 'بازجو', 'خبرنگار', 'صداو']
With bigrams: ['غلامحسین', 'محسنی_اژه', 'رییس', 'قوه_قضاییه', 'جمهوری_اسلامی', 'پاسخ', 'سوال', 'علی', 'رضوانی', 'بازجو', 'خبرنگار', 'صداو', 'سیما', 'درباره', 'نقش']


In [26]:
df.to_csv("../preprocess/predata.csv", index=False, encoding="utf-8-sig")

In [27]:
df = pd.read_csv("../preprocess/predata.csv")

df=df[df['clean_text'].str.len() > 20]
df = df.drop_duplicates("clean_text") 
docs = df['clean_text'].dropna().astype(str).tolist()

In [ ]:
# Sentence embeddings 
embedder = SentenceTransformer(
    "xmanii/maux-gte-persian",
    trust_remote_code=True
)

# Vectorizing 
vectorizer = CountVectorizer(
    tokenizer=persian_tokenizer,   
    token_pattern=None,            
    lowercase=False       
)

# Clustering knobs
umap_model = UMAP(n_neighbors=15, min_dist=0.1, metric="cosine")
hdb_model  = HDBSCAN(min_cluster_size=10, min_samples=5, prediction_data=True)

# Fit
topic_model = BERTopic(
    embedding_model=embedder,
    vectorizer_model=vectorizer,
    umap_model=umap_model,
    hdbscan_model=hdb_model,
    min_topic_size=10,
    calculate_probabilities=True,
)

topics, probs = topic_model.fit_transform(docs)
topic_model.get_topic_info().head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,0,1470,0_اسرائیل_جنگ_جمهوری_اسلامی_ایران,"[اسرائیل, جنگ, جمهوری_اسلامی, ایران, آمریکا, ح...",[نشست شورای امنیت ملی آمریکا که به بحث و بررسی...
1,1,34,1_آتش_جنگل_جنگلی_ترکیه,"[آتش, جنگل, جنگلی, ترکیه, گسترده, نشان, منطقه,...",[تصاویر هوایی، آتش‌سوزی گسترده‌ای را در نزدیکی...


In [42]:
topic_model.get_topic(0)

[('اسرائیل', 0.07321558290818024),
 ('جنگ', 0.0650711569512354),
 ('جمهوری_اسلامی', 0.06190402992594244),
 ('ایران', 0.05842963513723816),
 ('آمریکا', 0.03452387705704008),
 ('حمله', 0.026763040096664432),
 ('کشور', 0.02385149384545015),
 ('اعلام', 0.02259518312776282),
 ('جنگ_روزه', 0.02251486209134566),
 ('تهران', 0.02201829492406831)]

In [43]:
topic_model.visualize_topics()


ValueError: zero-size array to reduction operation maximum which has no identity